# Configure environment

In [ ]:
import os 
from dotenv import load_dotenv

load_dotenv()

tavily_key = os.getenv('TAVILY_API_KEY')
open_key = os.getenv('OPENAI_API_KEY')
print(tavily_key)
print(open_key)

# 3. Define the tool¶
Define the web search tool:

In [ ]:
from langchain_tavily import TavilySearch

tool = TavilySearch(max_results=5)
tools = [tool]
# tools.invoke("What is the capital of France?")

# 4. Define Graph and add nodes

In [ ]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("openai:gpt-4.1-mini")

In [ ]:
from typing import Annotated

from langchain_tavily import TavilySearch
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

tool = TavilySearch(max_results=2)
tools = [tool]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
# graph = graph_builder.compile()

# 1. Create a MemorySaver checkpointer¶
Create a MemorySaver checkpointer:

This is in-memory checkpointer, which is convenient for the tutorial. However, in a production application, you would likely change this to use SqliteSaver or PostgresSaver and connect a database.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

# 2. Compile the graph
Compile the graph with the provided checkpointer, which will checkpoint the State as the graph works through each node:

In [ ]:
graph = graph_builder.compile(checkpointer=memory)

# 3. Visualize the graph (optional)


In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

# 3. Interact with your chatbot¶
Now you can interact with your bot!

## 1. Pick a thread to use as the key for this conversation.

In [ ]:
config = {"configurable": {"thread_id": "1"}}

## 2. Call your chatbot:

In [ ]:
user_input = "Hi there! My name is Prasanta Narah."

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

## 4. Ask a follow up question
Toggle between following to check the memory
```
    config,
    # {"configurable": {"thread_id": "2"}},
```

In [ ]:
# user_input = "Remember my name? remember that there was two birds in my garden."
# user_input = "Tell my name and tell me how many birds were in my garden."
user_input = "What is the capital of France?"

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    # {"configurable": {"thread_id": "2"}},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

# 5. Inspect the state¶

By now, we have made a few checkpoints across two different threads. But what goes into a checkpoint? To inspect a graph's state for a given config at any time, call get_state(config).

In [ ]:
snapshot = graph.get_state(config)
snapshot

# COMBINED


In [ ]:
from typing import Annotated

from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

tool = TavilySearch(max_results=2)
tools = [tool]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)